<a href="https://colab.research.google.com/github/quantaosun/Gromacs-MD/blob/main/Gromacs_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# This notebook for a general solution simulation on cloud wtih Gromacs.

This notebook take http://www.mdtutorials.com/gmx/complex/ as a overall reference, the basic idea is to move this tutorial online as much as possible.

quantaosun@gmail.com created this notebook

Credit to the work of giribio/MDNotebooks as well, since the software
compilation method is borrowed there.

This notebook is still under editing, so maybe just follow my repository for a complete version later.







In [ ]:
#@title 1. Install Cmake
%%bash
lscpu
nvidia-smi
# Let's install a tool to get access to the ABFE input files from google drive

#Let us download the latest CMake - required for Gromacs 2021

wget https://github.com/Kitware/CMake/releases/download/v3.20.0-rc1/cmake-3.20.0-rc1.tar.gz
tar xfz cmake-3.20.0-rc1.tar.gz
## Install cmake

mkdir /content/cmake-3.20.0-rc1/build
cd /content/cmake-3.20.0-rc1/build
cmake /content/cmake-3.20.0-rc1/
make -j 2
make install
cd /content/
# Install open babel
wget https://github.com/openbabel/openbabel/archive/refs/tags/openbabel-3-1-1.tar.gz
tar -xzvf openbabel-3-1-1.tar.gz
cd openbabel-openbabel-3-1-1/
ls
mkdir build
cd build
cmake ../
make
make install
cd /content/
# Install netx 
!yes | pip uninstall networkx
!pip install networkx==2.3 

In [ ]:
#@title 2.Compile Gromacs
%%bash
wget ftp://ftp.gromacs.org/gromacs/gromacs-2021.tar.gz
tar xfz gromacs-2021.tar.gz
cd /content/gromacs-2021
mkdir /content/gromacs-2021/build
cd /content/gromacs-2021/build
cmake .. -DGMX_BUILD_OWN_FFTW=ON -DREGRESSIONTEST_DOWNLOAD=ON -DGMX_GPU=CUDA
make -j 2
make install
cd /content

In [ ]:
#@title 3.Input PDB ID you want to simulate
complex = "3HTB" #@param {type:"string"}
ligand = "JZ4" #@param {type:"string"}
pdb = complex + ".pdb"
LIGNAD = ligand + ".pdb"
#!/home/aistudio/external-libraries/bin/pdbfixer  '{pdb}' --ph=7  --replace-nonstandard --add-residues 
!!wget http://www.pdb.org/pdb/files/'{pdb}' > prot.pdb
!grep -v HOH '{pdb}' > prot_clean.pdb
!grep '{ligand}' prot_clean.pdb > '{ligand}'.pdb
!grep -v '{ligand}' prot_clean.pdb > prot_clean2.pdb
!grep -v PO4 prot_clean2.pdb > prot_clean3.pdb 
!grep -v BME prot_clean3.pdb > prot_clean4.pdb # here we drop all co-factors to have only protein itself

Beaware that the JZ4.pdb as well as the prot_clean4.pdb do not have any hydrogen atoms yet.

In [ ]:
#@title 4.Download protein topology file
!wget http://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/charmm36-mar2019.ff.tgz
!mv download.php?filename=CHARMM_ff_params_files%2Fcharmm36-mar2019.ff.tgz charmm36-mar2019.ff.tgz
!tar -zxvf charmm36-mar2019.ff.tgz
!rm charmm36-mar2019.ff.tgz
!pwd

In [ ]:
#@title 5.Select option 1(charmm36 force field) and optino 1 (TIP3 water) for next command
!source /usr/local/gromacs/bin/GMXRC
!echo "source /usr/local/gromacs/bin/GMXRC" >> ~/.bashrc
!source ~/.bashrc
#select option 1(charmm36 force field) and optino 1 (TIP3 water) for next command
!pwd
#Use pdb2gmx to convert pdb to gro file
!/usr/local/gromacs/bin/gmx pdb2gmx -f prot_clean4.pdb -o prot_pros.gro 
!tail topol.top
#Build a working directory for simulation jobs
!mkdir jobs
%cd jobs/

In [ ]:
#@title 6. Modify small molecule files
!obabel -ipdb JZ4.pdb -omol2 -O jz4.mol2 -h # convert the format and add hydrogen atoms
!sed -i 's/JZ4.pdb/JZ4/' jz4.mol2
!cat jz4.mol2
!wget http://www.mdtutorials.com/gmx/complex/Files/sort_mol2_bonds.pl
!perl sort_mol2_bonds.pl jz4.mol2 jz4_fix.mol2 
!cat jz4_fix.mol2 # pay attention to the bond order section

# Download the jz4_fix.mol2 and updoad to CGenff server Then paste the result to the mannually touched jz4.txt,then rename to jz4.str https://cgenff.umaryland.edu/userAccount/userLogin.php 


In [ ]:
#@title 7. Manipulate small molecule files
!touch jz4.txt # the only reason I use text format is it can be directly open upon double clik on the right.
!head jz4.txt
!mv jz4.txt jz4.str
!wget http://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/cgenff_charmm2gmx_py3_nx2.py
!mv download.php\?filename\=CHARMM_ff_params_files%2Fcgenff_charmm2gmx_py3_nx2.py cgenff_charmm2gmx_py3_nx2.py
#There might be an error, but that shouldn't influence this simulation job, so just ignore it.
!python cgenff_charmm2gmx_py3_nx2.py JZ4 jz4_fix.mol2 jz4.str charmm36-mar2019.ff
#After this, the output jz4_ini.pdb contains all the 22 atoms of the ligand JZ4
!/usr/local/gromacs/bin/gmx editconf -f jz4_ini.pdb -o jz4.gro
#Let's have a look at jz4.gro
!cat jz4.gro

Please be advised you double check the command line has acturally worked, some time you hit the triangle but the result maybe in a russ to write and save to the file as we here all deal with online things, if at 
that moment your connection is slow, then there is a chance you may fail the command.

In [ ]:
#@title 8.Combine protein and ligand
!cat prot_pros.gro > complex.gro
#Always have a look at the file you just produced,there should be 2614 atoms inside
!cat complex.gro
!tail -1 complex.gro > last_line.txt

!cat last_line.txt
!head -n -1 complex.gro > temp.txt

!cat temp.txt
#There still 2614 atoms inside temp.txt but the last line of box vector is moved out temperarily.
#Let's combine jz4.gro and complex.gro now
!cat jz4.gro
!grep JZ4 jz4.gro >> temp.txt

#Let's have a look at if the ligand atoms have been added after the protein atoms
!cat temp.txt
#Next let's add the box vector back to the last line.
!cat last_line.txt >> temp.txt
!head -10 temp.txt
!tail -30 temp.txt
#Finally we need to increase the atom number claim
!sed -i '2s/2614/2636/' temp.txt
!head -10 temp.txt
!cat temp.txt > complex.gro # I use cat instead of mv, just to keep the temp.txt copy in case later we need it.
!head complex.gro

In [ ]:
#@title 9.Fix the topology
!cat topol.top > topol.txt
!tail -25 topol.txt
!sed -i "/; Include water topology/ { N; s/; Include water topology\n/; Include ligand topology #include "jz4.itp"\n&/ }" topol.txt
#you shoud see there is one more line before **"Include water topology"** now you need to put "#inclue "jz4.inp" to the next line. Pay attention to the **" "**, since they are likely to be missed out.
!tail -25 topol.txt
#Please make sure you have put the following before 
#**Include water topology.**

#```
#; Include ligand topology 
#include "jz4.itp"
#```

!tail -25 topol.txt
!echo "JZ4           1" >> topol.txt
!tail -25 topol.txt
#Now you should notice there is one more line at the end


#```
#Protein_chain_A     1
#```
#manually insert the another block,

#```
#; Include ligand parameters
 #include "jz4.prm"
#```
#before [moleculetype]
!head -30 topol.txt
!cat topol.txt > topol.top # again, here I used cat instead of mv to keep topol.txt as an extra copy in case there might be an error later
#Let's double check the topol.top
!cat topol.top

In [ ]:
#@title 10. Gromacs simulation
!/usr/local/gromacs/bin/gmx editconf -f complex.gro -o newbox.gro -bt dodecahedron -d 10.0 
#After this, there shouldn't be any change in the topology file, since there is only a "box" was added to the gro file
#There should be a newbox.gro files was written as an output.
!tail topol.top
!/usr/local/gromacs/bin/gmx solvate -cp newbox.gro -cs spc216.gro -p topol.top -o solv.gro 
!tail topol.top
#Now you should see 10294 waters added to the topology, 
!wget http://www.mdtutorials.com/gmx/complex/Files/ions.mdp
!/usr/local/gromacs/bin/gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr

#**If you encounter an error here saying that you have different atom numbers in gro and top file, open the topol.top file to see if the JZ4 molecule type had bee actually added to the last block of the top file.**
#please select group 15:SOL for the next command
!/usr/local/gromacs/bin/gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -pname NA -nname CL -neutral 

!tail topol.top
#Now you should see the solvent have decreased a little bit and 6 more chloride ions added instead
!wget http://www.mdtutorials.com/gmx/complex/Files/em.mdp
!/usr/local/gromacs/bin/gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr
!/usr/local/gromacs/bin/gmx  mdrun -ntmpi 2 -ntomp 1 -v -deffnm em -nb gpu
# -ntmpi 2 -ntomp 1 -pin on -nb gpu (ommand improved)
# Make restrains


#```
## Input this for the next command
# > 0 & ! a H*
# > q
#```


!/usr/local/gromacs/bin/gmx make_ndx -f jz4.gro -o index_jz4.ndx
#select **"group 3"** for the next command
!/usr/local/gromacs/bin/gmx genrestr -f jz4.gro -n index_jz4.ndx -o posre_jz4.itp -fc 1000 1000 1000
# **next we need to modify the topology file again**
! cat topol.top > topol_2.txt
#copy and pase this before **"Include waters topology"**

#``` 
#; Ligand position restraints
##ifdef POSRES
##include "posre_jz4.itp"
##endif
#```
#I don't suggest you use next line command unless you know what it exactly will do, just copy and paste maybe. 
#!sed -i "/; Include water topology/ { N; s/; Include water topology\n/; Ligand position restraints ifdef POSRES include "posre_jz4.itp" endif\n&/ }" topol_2.txt
!tail -30 topol_2.txt
!cat topol_2.txt > topol.top 
#**Input this for the next command**
#```
#> 1 | 13
#> q
#```
#温度控制
!/usr/local/gromacs/bin/gmx make_ndx -f em.gro -o index.ndx

In [ ]:
%%bash
cd /content/
mkdir new
cd new